In [ ]:
%run yulesimon-lib.ipynb

# CONSTANTS

In [ ]:
N_RANGE=[11]; RANDOM_STATES=[17]
# N_RANGE=[5,8,11]; RANDOM_STATES=[0, 17]
# N_RANGE=range(5, 12); RANDOM_STATES=[0, 3, 5, 8, 11, 16, 17, 20, 21, 24]

trials_out = {}

PLOT_LEARNING_CURVES = True

RUN_DNN = True
RUN_DNN_ZEROS = True
RUN_DNN_WIN = True
RUN_DNN_WIN_ZEROS = True
RUN_CNN = True
RUN_CNN_ZEROS = True
RUN_CNNM = True


In [ ]:
# helper function to plot learning curves for the given trial output
def trial_plot_learning_curves(trial):
    history_path = '{}.history'.format(trial['base_path'])
    history = pickle.load(open(history_path, 'rb'))
    plot_learning_curves(history)

# DNN

In [ ]:
%%time
if RUN_DNN:
    nn = 'DNN'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=False, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES:
        trial_plot_learning_curves(trials_out[nn])
        print(trials_out[nn]['input_sample'])

# DNN zeros

In [ ]:
%%time
if RUN_DNN_ZEROS:
    nn = 'DNN_ZEROS'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# DNN win test
Test to show that 'data_hstack_win_sum()' is working correctly

In [ ]:
X_win_test = np.array([ [1,2,3,4,5,6,7], [1,2,3,4,5,6,7] ])

print('X_win_test:')
print(X_win_test)

OUT_win_test, window_sizes = data_hstack_win_sum(X_win_test)

print()
print('window_sizes:', window_sizes)

print()
print('OUT_win_test:')
print(OUT_win_test)

# DNN win

In [ ]:
%%time
if RUN_DNN_WIN:
    nn = 'DNN_WIN'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=False, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# DNN win zeros

In [ ]:
%%time
if RUN_DNN_WIN_ZEROS:
    nn = 'DNN_WIN_ZEROS'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# CNN

In [ ]:
%%time
if RUN_CNN:
    nn = 'CNN'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=False, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# CNN zeros

In [ ]:
%%time
if RUN_CNN_ZEROS:
    nn = 'CNN_ZEROS'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# CNN multi-layer

In [ ]:
%%time
if RUN_CNNM:
    nn = 'CNNM'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=False, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# CNN multi-layer + zeros

In [ ]:
%%time
if RUN_CNNM:
    nn = 'CNNM_ZEROS'
    trials_out[nn] = trial(nn=nn, N_range=N_RANGE, hstack_zeros=True, random_states=RANDOM_STATES)
    if PLOT_LEARNING_CURVES: trial_plot_learning_curves(trials_out[nn])

# plot log(N) vs sqrt_mse (all trials)

In [ ]:
def plot_sqrt_mse(plot_std=False):
    
    plt.figure(figsize=(15, 4))
    
    ax1 = plt.subplot(1, 2, 1)

    ax1.set(title='sqrt_mse', xlabel='log(N)', ylabel='sqrt_mse')

    colors = iter(cm.rainbow(np.linspace(0, 1, len(trials_out))))

    for i, out in enumerate(trials_out):
        x = np.log10(out['a_N'])
        y = out['a_sqrt_mse']
        c = next(colors)
        ax1.plot(x, y, linewidth=0.5, c=c, marker='o', markersize=3, label=out['nn'])
        if plot_std:
            std = out['a_std_abs_errors']
            ax1.fill_between(x, y - std, y + std, facecolor=colors[i], alpha=0.3)
    
    plt.legend(loc='upper right')

plot_sqrt_mse()


# print and save statistics (yulesimon.csv)

In [ ]:
dict_list = []
for t in trials_out:
    dict_list.append(('N', t['a_N']))
for t in trials_out:
    dict_list.append(('sqrt_mse_{}'.format(t['nn']), t['a_sqrt_mse']))
for t in trials_out:
    dict_list.append(('std_{}'.format(t['nn']), t['a_std_abs_errors']))
for t in trials_out:
    dict_list.append(('avg_abs_error_{}'.format(t['nn']), t['a_avg_abs_errors']))

df = pd.DataFrame(dict(dict_list))

df = df.set_index('N')

df.to_csv('yulesimon.csv', float_format='%.5f')

columns=[]
for t in trials_out:
    columns.append(('sqrt_MSE',t['nn']))
for t in trials_out:
    columns.append(('STD',t['nn']))
for t in trials_out:
    columns.append(('avg_abs_err',t['nn']))

df.columns=pd.MultiIndex.from_tuples(columns)

print('err = y_test - y_pred')
df